# Settle Up Problem (Dlužníček)

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

## Motivation

You went on a trip with a group of your friends. All of you shared some expenses, and now it is the time to settle all the debts. It is clear that everyone should pay the same amount; however, people are lazy, and so you want to find the solution which minimizes the number of transactions.

## Input

You are given the following:

* A set of people $P$
* For every person $i \in P$ the cost $c_i$ (i.e., amount of money that $i$ payed)

For the experiments, you may use the following instance:

In [1]:
!pip install gurobipy
import gurobipy as g

Defaulting to user installation because normal site-packages is not writeable


In [32]:
P = set(["A", "B", "C", "D"])
c_d = {"A": 0, "B": 590, "C": 110, "D": 300}  # c_i is accessed by calling c[i]
c = [0, 590, 110, 300]  # c_i is accessed by calling c[i]
sv = sum(c_d.values())/len(c)  # the settlement value

Number $sv$ (the settlement value) is the fair price that every person should pay.

## Output

You should find a list of tuples $(x_k, y_k, z_k)$ representing the transactions: person $x_k$ should pay person $y_k$ $z_k$ euros. The number of transactions (i.e., the length of the list) should be minimized.

For the given instance, the **optimal solution** consists of 3 transactions, namely

```
A -> B: 250.0
C -> D: 50.0
C -> B: 90.0
```

## Exercise

Implement the ILP model of the problem. You can assume that the settlement value is int (or was rounded).

In [43]:
import gurobipy as g  # import Gurobi module

# model --------------------------------------------------

m = g.Model()


# - ADD VARIABLES - jako promene zvolime transakce
# matice - na radcich, co jsem poslal; ve sloupcich, co jsem dostal
t_a = m.addVars(len(c), len(c), vtype=g.GRB.INTEGER)  # velikost transakce
t_h = m.addVars(len(c), len(c), vtype=g.GRB.BINARY)  # transakce probehla


# - ADD CONSTRAINTS
for person_id in range(len(c)):
    m.addConstr(t_a.sum(person_id, "*") -
                t_a.sum("*", person_id) + c[person_id] == sv)
    
for i in range(len(c)):
        for j in range(len(c)):
            m.addConstr(t_a[i,j] <= t_h[i,j] * sum(c))


# - SET OBJECTIVE
m.setObjective(t_h.sum() * 100000 + t_a.sum(), sense=g.GRB.MINIMIZE) # pridat vahu minimalizaci transakci, pote minimaluzujeme rozdily sum


# call the solver -------------------------------------------

m.optimize()


# print the solution -----------------------------------------

print('\nSOLUTION:')
print([t_a[i, j].x for i in range(len(c)) for j in range(len(c))])

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1145G7 @ 2.60GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 20 rows, 32 columns and 56 nonzeros
Model fingerprint: 0xc3454dda
Variable types: 0 continuous, 32 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 3e+02]
Found heuristic solution: objective 300790.00000
Presolve removed 4 rows and 8 columns
Presolve time: 0.00s
Presolved: 16 rows, 24 columns, 48 nonzeros
Variable types: 0 continuous, 24 integer (12 binary)

Root relaxation: objective 3.939000e+04, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

## Additional experiments

* experiment with different values of the 'big M' constant
* try to generalize the model to work even with float sv (e.g., rounded to 0.01)
